In [1]:
import pandas as pd
import re
import numpy as np
pd.set_option('display.max_columns', 100)  # or 1000
pd.set_option('display.max_rows', 1000)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199
from soynlp.hangle import levenshtein, jamo_levenshtein

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [2]:
lexicon = pd.read_csv('OCR_lexicon.csv', header=None, names=['가공식품'])

In [26]:
# lexicon[lexicon['preprocess'].apply(lambda x : bool(re.compile('!').search(x)))]

,가공식품,preprocess,ed


In [3]:
lexicon['preprocess'] = lexicon['가공식품'].apply(lambda x : x.replace(',', '_' ))
lexicon['preprocess'] = lexicon['preprocess'].apply(lambda x : x.replace(' ', '_' ))

In [4]:
lexicon['preprocess'] = lexicon['preprocess'].apply(lambda x : x.replace('(', '_' ))
lexicon['preprocess'] = lexicon['preprocess'].apply(lambda x : x.replace(')', '_' ))
lexicon['preprocess'] = lexicon['preprocess'].apply(lambda x : x.replace('[', '_' ))
lexicon['preprocess'] = lexicon['preprocess'].apply(lambda x : x.replace(']', '_' ))
lexicon['preprocess'] = lexicon['preprocess'].apply(lambda x : x.replace('!', '!_' ))
lexicon['preprocess'] = lexicon['preprocess'].apply(lambda x : x.replace('__', '_' ))

In [5]:
# 마지막에 _ 만 달랑 있는애들 _ 제거하기
lexicon['preprocess'] = lexicon['preprocess'].apply(lambda x : x[:-1] if bool(re.compile('.*_$').search(x)) else x )
# 앞에 _ 만 있는애들 _ 제거하기
lexicon['preprocess'] = lexicon['preprocess'].apply(lambda x : x[1:] if bool(re.compile('^_.*').search(x)) else x )

In [6]:
lexicon.to_csv('OCR_lexicon_pre.csv', index=False)

## SAVE POINT

In [19]:
lexicon = pd.read_csv('OCR_lexicon_pre.csv')
lexicon

,가공식품,preprocess
0,"비스켓, 패스트푸드",비스켓_패스트푸드
1,"비스켓 ,KFC",비스켓_KFC
2,"엔칠라다, 패스트푸드",엔칠라다_패스트푸드
3,애플파이_맥도날드,애플파이_맥도날드
4,애플파이_버거킹,애플파이_버거킹
...,...,...
2450,에그 베이글 스타벅스,에그_베이글_스타벅스
2451,베이컨 치즈베이글 스타벅스,베이컨_치즈베이글_스타벅스
2452,포크 커틀릿 샌드위치 스타벅스,포크_커틀릿_샌드위치_스타벅스
2453,만한대찬 우육면 매운 소고기맛 Uni President,만한대찬_우육면_매운_소고기맛_Uni_President


In [21]:
ed_list = []
for word in lexicon['preprocess']:
    word_ed = []
    word_split = word.split('_')
    for each in example:
    
        best_of_word = []
        for word_ in word_split:
            ed = levenshtein(each, word_)
            er = ed/(max(len(word_), len(each)))
            if er > 0.5:
                er= 1
            best_of_word.append(er)
        word_ed.append(min(best_of_word))
    print(f'word_ed : {word_ed}')
    n_1 = word_ed.count(1)
    if n_1 >1:
        for _ in range(n_1 -1):
            word_ed.remove(1)
        print('word_ed after pop : ', word_ed)
        div_val = len(word_split) - n_1 + 1
        print(f'div val : {div_val}')
        ed_list.append(sum(word_ed)/div_val)
    else:
        ed_list.append(sum(word_ed)/len(word_split))
#     ed_list.append(word_ed/len(word_split))

# lexicon['ed'] = ed_list

word_ed : [1, 1, 1]
word_ed after pop :  [1]
div val : 0


ZeroDivisionError: division by zero

In [17]:
example = ['김윤승갓'] 

In [18]:
ed_list = []
for word in lexicon['preprocess']:
    min_ed = []
    word_split = word.split('_')
    for each in example:
        each_ed = []
        for word_ in word_split:
            ed = levenshtein(each, word_)
            er = ed/(max(len(word_), len(each)))
            if er > 0.5:
                er= 1
            each_ed.append(er)
#             print(f'{each} -> {word_} : {er}')
        min_ed.append(min(each_ed))
    ed_list.append(sum(min_ed)/len(example))

lexicon['ed'] = ed_list

In [22]:
min_data = lexicon[lexicon['ed'] == lexicon['ed'].min()]
if min_data['ed'].min() >0.7:
    print('not in lexicon')
else:
    min_idx = min_data['preprocess'].apply(lambda x :  abs(len(example) - len(x.split('_')))).sort_values().index[0]
    print(why.loc[idx]['preprocess'])
why

not in lexicon


,가공식품,preprocess,ed
0,"비스켓, 패스트푸드",비스켓_패스트푸드,1.0
1,"비스켓 ,KFC",비스켓_KFC,1.0
2,"엔칠라다, 패스트푸드",엔칠라다_패스트푸드,1.0
3,애플파이_맥도날드,애플파이_맥도날드,1.0
4,애플파이_버거킹,애플파이_버거킹,1.0
...,...,...,...
2450,에그 베이글 스타벅스,에그_베이글_스타벅스,1.0
2451,베이컨 치즈베이글 스타벅스,베이컨_치즈베이글_스타벅스,1.0
2452,포크 커틀릿 샌드위치 스타벅스,포크_커틀릿_샌드위치_스타벅스,1.0
2453,만한대찬 우육면 매운 소고기맛 Uni President,만한대찬_우육면_매운_소고기맛_Uni_President,1.0


In [10]:
lexicon.head(100)

,가공식품,preprocess,ed
0,"비스켓, 패스트푸드",비스켓_패스트푸드,1.0
1,"비스켓 ,KFC",비스켓_KFC,1.0
2,"엔칠라다, 패스트푸드",엔칠라다_패스트푸드,1.0
3,애플파이_맥도날드,애플파이_맥도날드,1.0
4,애플파이_버거킹,애플파이_버거킹,1.0
5,체리파이_맥도날드,체리파이_맥도날드,1.0
6,체리파이_버거킹,체리파이_버거킹,1.0
7,씬크리스피_슈프림피자_피자헛,씬크리스피_슈프림피자_피자헛,1.0
8,씬크리스피_슈퍼슈프림피자_피자헛,씬크리스피_슈퍼슈프림피자_피자헛,1.0
9,씬크리스피_치즈피자_피자헛,씬크리스피_치즈피자_피자헛,1.0


In [37]:
ed_list = []
for word in why['preprocess']:
    word_ed = []
    word_split = word.split('_')
    print(word)
    for word_ in word_split:
        best_of_word = []
        for each in example :
            ed = levenshtein(each, word_)
            er = ed/(max(len(word_), len(each)))
            if er > 0.5:
                er= 1
            print(f'{word_} -> {each} : ED {ed}, ER {er}')
            best_of_word.append(er)
        print(f'{word_} -> {example} : best_of_word : {best_of_word}')
        word_ed.append(min(best_of_word))
    print('word_ed : ', word_ed)
    n_1 = word_ed.count(1)
    if n_1 >1:
        for _ in range(n_1 -1):
            word_ed.pop(1)
            print(f'after pop WORD_ED  {word_ed}')
            div_val = len(word_split) - n_1 + 1
            ed_list.append(sum(word_ed)/len(div_val))
    else:
        ed_list.append(sum(word_ed)/len(word_split))

소고기육포
소고기육포 -> 만한대장 : ED 5, ER 1
소고기육포 -> 매은 : ED 5, ER 1
소고기육포 -> 소고기맛 : ED 2, ER 0.4
소고기육포 -> ['만한대장', '매은', '소고기맛'] : best_of_word : [1, 1, 0.4]
word_ed :  [0.4]


In [20]:
ed_list = []
for word in why['preprocess']:
    word_ed = 0
    word_split = word.split('_')
    print(word)
    for each in example :
    
        each_best = []    
        for word_ in word_split:
            ed = levenshtein(each, word_)
            er = ed/(max(len(word_), len(each)))
            if er > 0.5:
                er= 1
            print(f'{word_} -> {each} : ED {ed}, ER {er}')
            each_best.append(er)
        print(f'{word_} -> {example} : each_list : {each_best}')
        print(min(each_best))
        word_ed += min(each_best)
    ed_list.append(word_ed/len(word_split))

소고기육포
소고기육포 -> 만한대장 : ED 5, ER 1
소고기육포 -> ['만한대장', '매은', '소고기맛'] : each_list : [1]
1
소고기육포 -> 매은 : ED 5, ER 1
소고기육포 -> ['만한대장', '매은', '소고기맛'] : each_list : [1]
1
소고기육포 -> 소고기맛 : ED 2, ER 0.4
소고기육포 -> ['만한대장', '매은', '소고기맛'] : each_list : [0.4]
0.4


---------------

# 다합치기

In [5]:
lexicon = pd.read_csv('OCR_lexicon_pre.csv')
lexicon

,가공식품,preprocess
0,"비스켓, 패스트푸드",비스켓_패스트푸드
1,"비스켓 ,KFC",비스켓_KFC
2,"엔칠라다, 패스트푸드",엔칠라다_패스트푸드
3,애플파이_맥도날드,애플파이_맥도날드
4,애플파이_버거킹,애플파이_버거킹
...,...,...
2450,에그 베이글 스타벅스,에그_베이글_스타벅스
2451,베이컨 치즈베이글 스타벅스,베이컨_치즈베이글_스타벅스
2452,포크 커틀릿 샌드위치 스타벅스,포크_커틀릿_샌드위치_스타벅스
2453,만한대찬 우육면 매운 소고기맛 Uni President,만한대찬_우육면_매운_소고기맛_Uni_President


In [16]:
lexicon['join'] = lexicon['preprocess'].str.replace('_', '')
lexicon

,가공식품,preprocess,join,ed
0,"비스켓, 패스트푸드",비스켓_패스트푸드,비스켓패스트푸드,1.250000
1,"비스켓 ,KFC",비스켓_KFC,비스켓KFC,1.666667
2,"엔칠라다, 패스트푸드",엔칠라다_패스트푸드,엔칠라다패스트푸드,1.111111
3,애플파이_맥도날드,애플파이_맥도날드,애플파이맥도날드,1.250000
4,애플파이_버거킹,애플파이_버거킹,애플파이버거킹,1.428571
...,...,...,...,...
2450,에그 베이글 스타벅스,에그_베이글_스타벅스,에그베이글스타벅스,1.111111
2451,베이컨 치즈베이글 스타벅스,베이컨_치즈베이글_스타벅스,베이컨치즈베이글스타벅스,1.000000
2452,포크 커틀릿 샌드위치 스타벅스,포크_커틀릿_샌드위치_스타벅스,포크커틀릿샌드위치스타벅스,1.000000
2453,만한대찬 우육면 매운 소고기맛 Uni President,만한대찬_우육면_매운_소고기맛_Uni_President,만한대찬우육면매운소고기맛UniPresident,0.720000


In [17]:
example = '소고기맛매운만한대찬'

In [18]:
ed_list = []
for word  in lexicon['join']:
    ed = levenshtein(example, word)
    er = ed/(len(word))
    ed_list.append(er)
lexicon['ed'] = ed_list

In [19]:
lexicon[lexicon['ed']== lexicon['ed'].min()]

,가공식품,preprocess,join,ed
763,"불고기스모크햄, 한성기업",불고기스모크햄_한성기업,불고기스모크햄한성기업,0.727273
1388,오뚜기카레(매운맛)_오뚜기,오뚜기카레_매운맛_오뚜기,오뚜기카레매운맛오뚜기,0.727273
2350,소고기버섯비빔밥 풀무원,소고기버섯비빔밥_풀무원,소고기버섯비빔밥풀무원,0.727273


##### 다합치기 : 순서가 바뀔경우 실패한다